In [18]:
import os
import shutil
import re
import itertools as it
text_dir = "E1/texts"
dest_dir = "E1/temp_texts"
def reset():
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    shutil.copytree(text_dir,dest_dir)

In [2]:
# jstor cover
# Every page except for the title and cover page had exactly one header line
# Page 00 was cover
# Page num at bottom in 1888-0-1, a couple other floating numbers we can get rid of
# only footnotes 1889 pages 1 and 24
# Title page:
    #1888: Get rid of top 5 lines
    #1889 get rid of top 2 lines, bottom 4

In [21]:
"""→for file in files:
→if page is ‘00’:
→remove page
"""
reset()
files = os.listdir(dest_dir)
files.sort()
for file in files:
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    if page==0:
        os.remove(os.path.join(dest_dir,file))

In [ ]:
"""→for page in pages:
get rid of jstor footer
→split text into lines
→strip whitespace from edges of each line
if not title page:
    Only Get everything after first line
    if length of last line is 1, delete that line
    if year = 1889 and page=24:
        get rid of the bottom line
else:
    if year is 1888:
        get rid of top 5 lines
    else:
        get rid of top 2, bottom 4 lines
join lines together
strip whole text
    
"""
files = os.listdir(dest_dir)
files.sort()
for file in files:
    if file[0]=='.':
        continue
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    path = os.path.join(dest_dir,file)
    
    text = open(path,'r').read()
    jstor_split = text.split("This content downloaded from")
    text =jstor_split[0].strip()
    lines = text.split("\n")
    lines = [l.strip() for l in lines]
    if page!=1:
        save_lines = lines[1:]
        if len(save_lines[-1])==1 or (year=='1889' and page==24):
            print(f"Deleting line with '{save_lines[-1]}'")
            save_lines = save_lines[:-1]
    elif year=='1888': # get the first page on 1888
        #print(f"First page in 1888: {file}")
        save_lines = lines[5:]
    else: # only one left is 1889
        #print(f"First page for 1889: {file}")
        save_lines = lines[2:-3]
    text = "\n".join(save_lines)
    with open(path,'w') as f:
        f.write(text)

In [23]:
### Deal with line breaks across pages
files = os.listdir(dest_dir)
files.sort()
for first,second in it.pairwise(files):
    if not first.split("-")[:3]==second.split('-')[:3]:
        continue
    path1 = os.path.join(dest_dir,first)
    text1 = open(path1,'r').read()
    if text1[-1]=="-":
        #print(first)
        #print(text1.strip().split("\n")[-1])
        path2 = os.path.join(dest_dir,second)
        text2 = open(path2,'r').read()
        first_word = re.search("^([^\s]+)\s",text2).group(1)
        new_text2 = re.sub("^[^\s]+\s","",text2)
        #print(first_word)
        
        new_text1 = text1[:-1]+first_word
        open(path1,'w').write(new_text1.strip())
        open(path2,'w').write(new_text2.strip())

In [24]:
### remove dash errors
def fix_dash_errors(text):
    new_text = re.sub(r"([a-zA-Z]+)\-+\n([a-zA-Z]+)([^\w\n\s])?", # Captures 3 groups: first half of word, second half of word, optional punctuation
                      r"\1\2\3\n", #removes dash and moves line break
                      text)
    new_text_lines_stripped=[line.strip() for line in new_text.split('\n')] #remove any extra leading or trailing whitespace
    return "\n".join(new_text_lines_stripped) #join lines back together
files = os.listdir(dest_dir)
files.sort()
for file in files:
    if file[0]=='.':
        continue
    path = os.path.join(dest_dir,file)
    text = open(path,'r').read()
    new_text = fix_dash_errors(text)
    open(path,'w').write(new_text)